# Pravilnost programov


`Specifikacija`: opis/zahteva, kaj naj bi program delal

`Implementacija`: koda, ki jo napišemo, da bi zadostili specifikaciji


`Hoarove trojke`:

    ```
    P = precondition (predpogoj)
    C = koda
    Q = post condition (končni pogoj)

    { P } C { Q}
    ```

P, Q logični formuli, ki govorita o vrednostih spremenljivk.

Ko preverjamo pravilnost, nalogo razdelimo:

1. Ali se program ustavi?
2. Če predpostavimo, da se program ustavi, ali zadošča dani specifikaciji?


**Delna pravilnost**: `{ P } c { Q }`, če velja P in če se c ustavi (predpostavka) po izvedbi c velja Q

**Popolna pravilnost** `[ P ] c [ Q ]`, če velja P, potem se c ustavi (sklep) in po izvedbi c velja Q

Primer:

```
{ x = m ∧ y = n }   // P specifikacija
z := x;
x := y;
y := z;
{ x = n ∧ y = m }
```

Uporabimo **duhove** (ghost variable):
- m, n se v kodi ne smeta pojaviti


```
{ n >= 1}
s := n * (n=1) / 2
{ s = 1 + 2 + 3 + ... + n}
```

Primer

```
{ x <= 7}
{ x + 3 <= 10}
x := x + 3
{ x <= 10 }
```

## Pravila sklepanja

Pravila sklepanja, H1, ... HN, hipoteze, S sklep

```
H1, H2, ..., Hn
---------------
      S
```

### Pogojni stavek

- Pravilo za pogojni stavek skripta

```
{ true }
if x < 5 then
    { x < 5 }
    y := x
    { x < 5 ∧ y = x }
else
    { x >= 5 }
    y = 0
    { x >= 5 ∧ y = 0 }
end
{ y < 5 }


### Zanka while

P je invarianta zanke (se ne spreminja). Poiščemo neko količino, ki se zmanjšuje a se ne more zmanjševati v nedogled

```
{P ∧ Q } c { P }
----------------
{ P } while b do c done { P ∧ ¬b }
```


```
[ b ≥ 0 ]
i := 0;
p := 1;
while i < b do
    p := p * a;
    i := i + 1
done
[ p = a^b]
```


1. Dokazi delno pravilnost

    ```
    { b ≥ 0 }

    i := 0;
    { b ≥ 0, i = 0 }
    p := 1;
    { b>=0, i=0, p=1 }
    INVARIANTA: { p = a^i, i ≤ b } (moram spraviti skozi zanko)
    while i < b do
        { p = a^i, i ≤ b ∧ i < b } 1. Začetek zanke varianta
        { p*a = a^{i+1}, i+1 ≤ b}
        p := p * a;
        { p = a^{i=1} ∧ i+1 ≤ b}
        i := i + 1
        { p = a^i, i ≤ b } 2. Konec zanke varianta (od spodaj navzgor gremo je lazje)
    done
    { p = a^i, i ≤ b ∧ ¬(i < b) }
    { p = a^i, i ≤ b ∧ i ≥ b }
    { p = a^i, i = b }

    { p = a^b }
    ```

    - Recept za invarianto, naredimo tabelo za loop:
        - rabimo b vključiti v invarianto (ker nastopa kot pogoj v while)

    ```
    | i | p |
    |- -|- -|
    | 0 | 1 |
    | 1 | a |
    | 2 | a^2 |
    | 3 | a^3 |

    Vidimo da je p = a^i, rabimo še i ≤ b
    ```

2. Dokazi **popolno pravilnost**
    - količina `d = b - i` se zmanjšuje in je ≥ 0, d nemore bit negativen ker je `i ≤ b` (v invarianti)
    - dejstvo ` b - i ≥ 0` sledi iz i ≤ b, kar smo preverili (je del invariante)

    ```
    # Količina b-i se zmanjšuje v telesu zanke
        [ p = a^i, i ≤ b ∧ i < b, b - i = d ]
        [ p*a = a^{i+1}, i+1 ≤ b, b - i = d]
        p := p * a;
        [ p = a^{i=1} ∧ i+1 ≤ b, b - i = d]
        [ p = a^{i=1} ∧ i+1 ≤ b, b - (i+1-i) = d]
        i := i + 1
        [ p = a^i, i ≤ b, b - (i-1) = d]
        [ p = a^i, i ≤ b, b - i = d - 1 ]
        [ p = a^i, i ≤ b, b - i = d - 1 < d ]

        [ p = a^i, i ≤ b, b - i < d] # Dokazali smo da se količina zmanjša
    ```
